In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from brainviewer import NapariBrainViewer, cm_inferno_alpha

# Working With Point Clouds

this section show examples of how to work with point clouds of neurons and their respective activities.

## Generating very very bad fake neural activity

In [4]:
import numpy as np
from scipy.spatial import distance_matrix

In [166]:
# parameters of the simulation
space_size = 100
s_xyz = np.array([space_size,space_size,space_size])

n_regions = 15
n_times = 3000
n_neurons = 5000

In [167]:
# creating regions

## spatial
com_regions = np.random.uniform(low=0*s_xyz, high=s_xyz, size=(n_regions, 3)) # centers of masse


## temporal
p_join_regions = np.random.choice( # probabilities of 2 regions activating simultaneously
    [0,1,0.1,0.9,0],
    p = [0.3,0.3,0.175,0.175,0.05],
    size=(n_regions, n_regions),
)
p_join_regions = ( p_join_regions + p_join_regions.conj().T )/2

region_acts = np.zeros((n_times, n_regions)) # random telegraph signal for each region with p_join contraint
p = 0.001
ref_j = 0
for i in range(1,n_times):
    if np.random.rand() < p*2:
        ref_j = np.random.randint(0,n_regions)
    if np.random.rand() < p:
        region_acts[i:, ref_j] = 1-region_acts[i-1, ref_j]
    for j in range(n_regions):
        if j == ref_j:
            continue
        if np.random.rand() < p_join_regions[ref_j,j]:
            region_acts[i:,j] = region_acts[i:,ref_j]
        else:
            region_acts[i:,j] = 1-region_acts[i:,ref_j]

In [168]:
# creating neurons

## spatial
com_neurons = np.random.uniform(low=0*s_xyz, high=s_xyz, size=(n_neurons, 3)) # centers of masse


## participation of neurons in each region
dists = distance_matrix(com_neurons, com_regions)
def norm_tanh(x, x0=20, width=10):
    xmin, xmax = x.min(), x.max()
    x = (x-xmin) / (xmax - xmin)
    x0 = (x0-xmin) / (xmax - xmin)
    width = (width-xmin) / (xmax - xmin)
    y = (1 - np.tanh((x-x0)/width*np.pi))/2
    return y/y.max()
participation_factor = norm_tanh(dists)

## temporal
neuron_tendencies = region_acts@participation_factor.T
neuron_spikes = np.random.poisson(neuron_tendencies*5)

## Opening the brain viewer

In [169]:
nbv = NapariBrainViewer()

## Scatter Neurons

In [170]:
# layer_com = nbv.points(com_neurons, blending="translucent_no_depth", size=2)

## Scatter neurons with single frame activity

In [171]:
# layer_activity = nbv.points(
#     com_neurons, neuron_spikes[50,:], 
#     cmap=cm_inferno_alpha, crange=(0,+5), 
#     blending="translucent_no_depth",
#     size=2,
# )

## Display neuron activity as interactive line plot

In [172]:
from brainviewer import ActivityViewer, PointLayerSelector

In [173]:
# av = ActivityViewer(nbv)
# pls = PointLayerSelector(layer_com, av, neuron_spikes)

## Display interactive correlation maps 

In [174]:
from brainviewer import PointLayerPairwise, cm_seismic_alpha

In [175]:
C = np.corrcoef(neuron_spikes.T) # correlations between neurons
C[~np.isfinite(C)] = 0;

/home/ljp/.local/bin/miniforge3/envs/brainviewer-env/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/ljp/.local/bin/miniforge3/envs/brainviewer-env/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [176]:
# plp = PointLayerPairwise(layer_com, C, cm_seismic_alpha, (-1,+1))

## Interactive coupling maps with region selection

### Selection of regions

In [177]:
import h5py
from brainviewer import enable_selection

file_path = '/home/ljp/Documents/Training/combined.h5'
file = h5py.File(file_path, "r")
coords = file['Data/Brain/coords']

brain_layer = nbv.points(coords[()], size=4, name="Neurons")
# brain_layer = layer_com

In [178]:
couplings = np.load('/home/ljp/Documents/Training/couplings.npy')
# couplings = C

In [179]:
enable_selection(nbv, brain_layer, couplings, cm_seismic_alpha, (0, 1e-3*couplings.max()))

### Grid on fish brain

In [180]:
import pandas as pd
grid = pd.read_csv('/home/ljp/Science/CalibTotalPatterns.csv')

In [181]:
grid['x_um_cor'] = 591-grid.y_um
grid['y_um_cor'] = 591-grid.x_um

In [182]:
affine = np.array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, 4.00000000e+01],
                   [ 0.00000000e+00, -2.78760308e-01,  1.06813412e+00, 3.48659345e+02],
                   [ 0.00000000e+00, -1.06813412e+00, -2.78760308e-01, 6.54845239e+02],
                   [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, 1.00000000e+00]])

In [183]:
nbv.load_nrrd('/home/ljp/Science/image.nrrd', affine=affine, name='NRRD')

<Image layer 'NRRD' at 0x74cade656cf0>

In [184]:
grid_points = np.array(grid[['y_um_cor', 'x_um_cor', 'z_um', ]])
grid_points[:,2] = 140
grid_layer = nbv.points(grid_points, affine=affine, name='Grid')

In [185]:
grid_layer.data

array([[140.  , 283.91, 230.76],
       [140.  , 261.67, 243.76],
       [140.  , 277.27, 220.64],
       ...,
       [140.  , 396.87, 333.89],
       [140.  , 421.13, 348.91],
       [140.  , 399.18, 361.62]])